In [99]:
import pandas as pd 
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction import *
from sklearn.metrics.pairwise import cosine_similarity # 사이킷 런에서 코사인 유사도 함수 import 
from sklearn.feature_extraction.text import *

In [100]:
ratings = pd.read_csv("ratings.csv")
movies = pd.read_csv("movies.csv")


In [101]:
movies['genres'] = movies['genres'].apply(lambda x :x.replace('|',' '))
Id_year=movies[movies['title'].str.contains('[(0_9)]+')].iloc[:,:2]
Id_year['years'] = Id_year['title'].str.split().apply(lambda x:x[-1])

In [102]:
Id_year=Id_year.drop(columns='title')

In [103]:
Id_genres = movies[['movieId','genres']]

In [104]:
Id_genres['title'] = np.where(movies['title'].str.contains('[(0_9)]+'),movies['title'].str.split().apply(lambda x:x[:-1]).apply(' '.join),movies['title'])

In [105]:
new_movies = pd.merge(Id_genres,Id_year,on= 'movieId',how='outer')
new_movies

,movieId,genres,title,years
0,1,Adventure Animation Children Comedy Fantasy,Toy Story,(1995)
1,2,Adventure Children Fantasy,Jumanji,(1995)
2,3,Comedy Romance,Grumpier Old Men,(1995)
3,4,Comedy Drama Romance,Waiting to Exhale,(1995)
4,5,Comedy,Father of the Bride Part II,(1995)
...,...,...,...,...
9120,162672,Adventure Drama Romance,Mohenjo Daro,(2016)
9121,163056,Action Adventure Fantasy Sci-Fi,Shin Godzilla,(2016)
9122,163949,Documentary,The Beatles: Eight Days a Week - The Touring Y...,(2016)
9123,164977,Comedy,The Gay Desperado,(1936)


In [106]:
new_movies['years'].fillna('Unknown',inplace= True)

In [107]:
data = pd.merge(ratings,new_movies,on='movieId')
data

,userId,movieId,rating,timestamp,genres,title,years
0,1,31,2.5,1260759144,Drama,Dangerous Minds,(1995)
1,7,31,3.0,851868750,Drama,Dangerous Minds,(1995)
2,31,31,4.0,1273541953,Drama,Dangerous Minds,(1995)
3,32,31,4.0,834828440,Drama,Dangerous Minds,(1995)
4,36,31,3.0,847057202,Drama,Dangerous Minds,(1995)
...,...,...,...,...,...,...,...
99999,664,64997,2.5,1343761859,Action Sci-Fi,War of the Worlds,(2005)
100000,664,72380,3.5,1344435977,Drama Horror Mystery Sci-Fi Thriller,"Box, The",(2009)
100001,665,129,3.0,995232528,Comedy Romance,Pie in the Sky,(1996)
100002,665,4736,1.0,1010197684,Comedy Drama Romance,Summer Catch,(2001)


In [108]:
movie_data = data.pivot_table('rating',index='movieId',columns='userId').fillna(0)

In [121]:
movie_data.shape

(9066, 671)

In [109]:
gen = CountVectorizer(ngram_range=(1,3))
c_gen = gen.fit_transform(new_movies['genres'])

In [110]:
gen.vocabulary_

{'adventure': 92,
 'animation': 176,
 'children': 227,
 'comedy': 269,
 'fantasy': 438,
 'adventure animation': 93,
 'animation children': 177,
 'children comedy': 228,
 'comedy fantasy': 293,
 'adventure animation children': 94,
 'animation children comedy': 178,
 'children comedy fantasy': 231,
 'adventure children': 102,
 'children fantasy': 249,
 'adventure children fantasy': 105,
 'romance': 556,
 'comedy romance': 320,
 'drama': 388,
 'comedy drama': 283,
 'drama romance': 423,
 'comedy drama romance': 289,
 'action': 0,
 'crime': 330,
 'thriller': 573,
 'action crime': 44,
 'crime thriller': 368,
 'action crime thriller': 53,
 'action adventure': 1,
 'adventure thriller': 168,
 'action adventure thriller': 12,
 'horror': 489,
 'comedy horror': 304,
 'adventure romance': 162,
 'action adventure romance': 10,
 'crime drama': 334,
 'action comedy': 31,
 'comedy crime': 270,
 'drama thriller': 430,
 'action comedy crime': 32,
 'comedy crime drama': 271,
 'crime drama thriller': 343,

In [111]:
c_gen.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [112]:
movieidtile=data[['movieId','title']].drop_duplicates()
movieidtile.head()

,movieId,title
0,31,Dangerous Minds
42,1029,Dumbo
84,1061,Sleepers
117,1129,Escape from New York
165,1172,Cinema Paradiso (Nuovo cinema Paradiso)


In [123]:
csmm = cosine_similarity(movie_data,movie_data)
dfcsmm=pd.DataFrame(csmm,columns = movieidtile['title'].values,index = movieidtile.movieId.values)
dfcsmm.head()

,Dangerous Minds,Dumbo,Sleepers,Escape from New York,Cinema Paradiso (Nuovo cinema Paradiso),"Deer Hunter, The",Ben-Hur,Gandhi,Dracula (Bram Stoker's Dracula),Cape Fear,...,Aloha,Entourage,Magic Mike XXL,7 Days in Hell,Pathology,War of the Worlds,"Box, The",Pie in the Sky,Summer Catch,"6th Man, The (Sixth Man, The)"
31,1.000000,0.394511,0.306516,0.133614,0.245102,0.377086,0.278629,0.063031,0.117499,0.310689,...,0.055829,0.031902,0.079755,0.079755,0.079755,0.079755,0.079755,0.0,0.0,0.055829
1029,0.394511,1.000000,0.217492,0.164651,0.278476,0.222003,0.207299,0.223524,0.113669,0.418124,...,0.000000,0.055038,0.068797,0.082557,0.082557,0.137594,0.068797,0.0,0.0,0.000000
1061,0.306516,0.217492,1.000000,0.177012,0.370732,0.247499,0.435648,0.127574,0.306717,0.191255,...,0.000000,0.000000,0.000000,0.116226,0.116226,0.000000,0.000000,0.0,0.0,0.000000
1129,0.133614,0.164651,0.177012,1.000000,0.179556,0.072518,0.184626,0.501513,0.254630,0.111447,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
1172,0.245102,0.278476,0.370732,0.179556,1.000000,0.272645,0.388476,0.194113,0.367941,0.246846,...,0.000000,0.176845,0.000000,0.117897,0.117897,0.000000,0.000000,0.0,0.0,0.000000


In [124]:
dfcsmm['Toy Story'].sort_values(ascending = False)

1       1.000000
2124    0.498970
3147    0.484860
317     0.477907
1011    0.468521
          ...   
2168    0.000000
2206    0.000000
2491    0.000000
2500    0.000000
3835    0.000000
Name: Toy Story, Length: 9066, dtype: float64

In [125]:
simindex = dfcsmm['Toy Story'].sort_values(ascending = False)[1:11]

In [126]:
movie_data.loc[1:]

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,4.0,0.0,...,0.0,4.0,3.5,0.0,0.0,0.0,0.0,0.0,4.0,5.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161944,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
162376,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
162542,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [127]:
# 제공한 영화와 가장 유사한 평점을 준 영화 5개 추천
def cos_sim(data,title_name,movieId_data,per = 0.1):
    movieId = movieId_data[movieId_data['title'] == title_name]['movieId'].values[0]
    lis_cos =[]
    nor = np.linalg.norm(data.loc[movieId,:])
    for j in range(len(data.index)):
        cos = (data.loc[movieId,:]@data.iloc[j,:])/(nor*np.linalg.norm(data.iloc[j,:]))
        if (cos > per) and( data.index[j] != movieId) :
            lis_cos.append((cos,data.index[j]))
    return sorted(lis_cos,reverse=True)

In [128]:
recom_ratings = cos_sim(movie_data,'Toy Story',new_movies)
show_recom_ratings = pd.DataFrame([],columns = new_movies.columns)
for i in recom_ratings:
    show_recom_ratings = show_recom_ratings.append(new_movies[new_movies['movieId'] == i[1]])

In [129]:
recom_ratings

[(0.5947098120318489, 3114),
 (0.576187845976566, 260),
 (0.5645338614526411, 356),
 (0.5629456002603211, 780),
 (0.5480230213992765, 1265),
 (0.5367002895741999, 1270),
 (0.5351970778368947, 480),
 (0.5326850987406373, 4306),
 (0.529334044308981, 1210),
 (0.5268615741598919, 296),
 (0.5234293660370452, 2571),
 (0.5211224677355181, 1580),
 (0.5201349521307311, 4886),
 (0.5197331963653228, 2355),
 (0.5185603741742433, 318),
 (0.5143471364270416, 364),
 (0.5125441657615241, 588),
 (0.508723606959372, 2762),
 (0.5059754109061483, 1198),
 (0.5046628499601937, 1196),
 (0.502590463503237, 8961),
 (0.5009200410460862, 6377),
 (0.4981681555905294, 1073),
 (0.4964664532169273, 648),
 (0.49568997812323057, 593),
 (0.49552356089989047, 5952),
 (0.4938264624714002, 4993),
 (0.4892746610261179, 32),
 (0.4884886546635136, 608),
 (0.4842322203620194, 34),
 (0.47619837175776747, 1682),
 (0.4751611880935025, 2028),
 (0.47168594302817907, 7153),
 (0.47118806932903995, 595),
 (0.47002436060604286, 6539),

In [130]:
show_recom_ratings[:10]

,movieId,genres,title,years
2506,3114,Adventure Animation Children Comedy Fantasy,Toy Story 2,(1999)
232,260,Action Adventure Sci-Fi,Star Wars: Episode IV - A New Hope,(1977)
321,356,Comedy Drama Romance War,Forrest Gump,(1994)
644,780,Action Adventure Sci-Fi Thriller,Independence Day (a.k.a. ID4),(1996)
1019,1265,Comedy Fantasy Romance,Groundhog Day,(1993)
1024,1270,Adventure Comedy Sci-Fi,Back to the Future,(1985)
427,480,Action Adventure Sci-Fi Thriller,Jurassic Park,(1993)
3419,4306,Adventure Animation Children Comedy Fantasy Ro...,Shrek,(2001)
966,1210,Action Adventure Sci-Fi,Star Wars: Episode VI - Return of the Jedi,(1983)
266,296,Comedy Crime Drama Thriller,Pulp Fiction,(1994)


# 유사 평점 + 관련 장르

In [16]:
gen = CountVectorizer(ngram_range=(1,2)).fit_transform(new_movies['genres'])

In [17]:
gen_arr = gen.toarray()
gen_arr = pd.DataFrame(gen_arr,index = new_movies['movieId'])

In [18]:
# gen_arr = gen.toarray()
# 추천해준 영화가 제시된 영화와 장르가 유사한가
def find_sim(data,title_name,movieId_data,data2,func = cos_sim):
    try:
        pers =[0.7,0.4]
        for i in pers:
            list_ratings = cos_sim(data2,title_name,new_movies,i)
            list_gens = cos_sim(data,title_name,new_movies,i)
            show_recom_ratings_gen =pd.DataFrame([],columns = new_movies.columns)
            for i in list_ratings:
                for j in list_gens:
                    if i[1] == j[1]:
                        show_recom_ratings_gen = show_recom_ratings_gen.append(new_movies[new_movies['movieId'] == i[1]])
            if show_recom_ratings_gen.size != 0:
                return show_recom_ratings_gen
                break
        if show_recom_ratings_gen.size == 0:
            return 'similarity under 0.4'        
    except Exception as e:
        print('오류',e)


In [19]:
# 'xXx: State of the Union','Box, The','Toy Story','Jumanji'
find_sim(gen_arr,test_m[3],new_movies,movie_data)

,movieId,genres,title,years
527,595,Animation Children Fantasy Musical Romance IMAX,Beauty and the Beast,(1991)
137,158,Adventure Children,Casper,(1995)


In [20]:
# 유저 아이디 입력시 유사 유저가 본 영화 추천

In [21]:
user_matrix= data.pivot_table('rating',index='userId',columns='movieId').fillna(0)
user_matrix

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
669,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
cos_user = cosine_similarity(user_matrix,user_matrix)

In [25]:
user_df= pd.DataFrame(cos_user,index = user_matrix.index,columns = user_matrix.index)

In [24]:
user_id = int(input('유저 검색 시 다른 유저가 봤던 영화 추천')) # 검색 유저는 보지 않은 것들만 추천

recom_user = user_df.loc[user_id,:][user_df.loc[user_id,:] > 0].sort_values()[::-1][1:6].index # 5명 추천
reco_movie_df = pd.DataFrame([],columns=new_movies.columns)

list_recom_movie =[]
list_recom_movie.extend(user_matrix.loc[user_id,:][user_matrix.loc[user_id,:] > 0].index.values)

for i in recom_user:
    for j in user_matrix.loc[i][user_matrix.loc[i] > 4].index.values: # 평점 4점 이상:
        if j not in list_recom_movie:
            list_recom_movie.append(j)
            reco_movie_df = reco_movie_df.append(new_movies[new_movies['movieId'] == j])
reco_movie_df

,movieId,genres,title,years
302,337,Drama,What's Eating Gilbert Grape,(1993)
284,318,Crime Drama,"Shawshank Redemption, The",(1994)
20,21,Comedy Crime Thriller,Get Shorty,(1995)
31,32,Mystery Sci-Fi Thriller,Twelve Monkeys (a.k.a. 12 Monkeys),(1995)
282,316,Action Adventure Sci-Fi,Stargate,(1994)
294,329,Adventure Drama Sci-Fi,Star Trek: Generations,(1994)
384,435,Comedy Sci-Fi,Coneheads,(1993)
526,594,Animation Children Drama Fantasy Musical,Snow White and the Seven Dwarfs,(1937)
5,6,Action Crime Thriller,Heat,(1995)
15,16,Crime Drama,Casino,(1995)
